In [104]:
import tensorflow as tf
import pandas as pd
import string
from nltk import word_tokenize
from tensorflow import keras

import numpy as np

In [13]:
# fake = pd.read_csv('News _dataset/Fake.csv', encoding='utf-8')
# true = pd.read_csv('News _dataset/True.csv' , encoding='utf-8')

# fake['label'] = 0
# true['label'] = 1

# fake.head()
# data = pd.concat([fake,true], ignore_index=True)
# data.head()



In [83]:
# data.drop(['title', 'subject', 'date'], axis='columns', inplace=True)
# data = data.sample(frac = 1).reset_index(drop=True)
# print(data.tail())
# print(data.head())
# data.to_csv('data.csv')

data = pd.read_csv('data.csv',encoding='latin1')
data

,Unnamed: 0,text,label
0,0,"GAZA/CAIRO (Reuters) - Palestinian factions, i...",1
1,1,HARARE (Reuters) - Zimbabwean police arrested ...,1
2,2,Ronna Romney McDaniel is the Chairman of the M...,0
3,3,WASHINGTON (Reuters) - A small group of Republ...,1
4,4,"THE HUNTINGTON BEACH, CA RALLY WAS PRETTY BIG ...",0
...,...,...,...
44893,44893,There are two small problems with your analogy...,0
44894,44894,Consortium News Exclusive: The U.S. mainstream...,0
44895,44895,Just as drug addicts and alcoholics tend to be...,0
44896,44896,WASHINGTON (Reuters) - Republican presidential...,1


In [84]:
seed = 1337
rng = np.random.RandomState(seed)


# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(data))
train_samples = data['text'][:-num_validation_samples]
val_samples = data['text'][-num_validation_samples:]
train_labels = data['label'][:-num_validation_samples]
val_labels = data['label'][-num_validation_samples:]

    

In [85]:
Length_of_input_sentenses=500
# matrix of pretrained weights


from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

vectorizer = TextVectorization(max_tokens=50000, output_sequence_length=500)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples)
vectorizer.adapt(text_ds)


In [90]:
# Customize get_vocabulary method
def _get_vocabulary():
    keys, values = vectorizer._index_lookup_layer._table_handler.data()
    return [x.decode('utf-8', errors='ignore') for _, x in sorted(zip(values, keys))]

In [95]:
# vectorizer.get_vocabulary()
_get_vocabulary()[:5]

['', 'the', 'to', 'of', 'a']

In [99]:
output = vectorizer([["the cat sat on the mat"]])
output.numpy()[0, :6]

array([    2,  8988,  3266,     9,     2, 17253], dtype=int64)

In [97]:
# voc = vectorizer.get_vocabulary()
voc = _get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))



embeddings_index = {}
with open('glove.6B.100d.txt', encoding="utf8") as f:
    for line in f:
        
        word, coefs = line.split(maxsplit=1)
        
        coefs = np.fromstring(coefs, "f", sep=" ")
        
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [100]:
# X = tf.keras.layers.Embedding(input_dim=vocab_size,
#                             output_dim=100,
#                             input_length=Length_of_input_sequences,
#                             embeddings_initializer=matrix_of_pretrained_weights
#                             )


In [101]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 38238 words (11761 misses)


In [105]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

# Model